In [5]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
import csv
import matplotlib.pyplot as plt

In [46]:
baseurl = "https://www.bajajallianz.com/general-insurance.html"

In [47]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'}

In [48]:
pip install httplib2

Note: you may need to restart the kernel to use updated packages.


In [49]:
import httplib2


In [50]:
http = httplib2.Http()

response, content = http.request(baseurl)

links = []
title = []


In [51]:
soup = BeautifulSoup(content)
for link in soup.findAll("a",attrs={'href': re.compile("^https://")}):
    links.append(str(link.attrs['href']))
    title.append(str(link.get('title')))

In [52]:
df = pd.DataFrame(list(zip(links,title)), columns =['LINK', 'TITLE'])

In [42]:
df.to_csv('bajaj_insurance_datasets.csv')


In [53]:
df.shape

(104, 2)

In [54]:

df.head(10)

,LINK,TITLE
0,https://www.bajajallianz.com/about-us/customer...,Support
1,https://www.bajajallianz.com/about-us.html,About Us
2,https://jobs.bjaz.in/#!/,Careers
3,https://www.bajajallianz.com/about-us/become-g...,Become An Agent
4,https://www.bajajallianz.com/about-us/general-...,Awards
5,https://www.bajajallianz.com/motor-insurance.html,Motor Insurance
6,https://www.bajajallianz.com/motor-insurance/c...,Car Insurance
7,https://www.bajajallianz.com/motor-insurance/t...,Two Wheeler Insurance
8,https://www.bajajallianz.com/motor-insurance/l...,Long Term Two Wheeler Insurance
9,https://www.bajajallianz.com/motor-insurance/t...,Third Party Car Insurance


# word2vec using para dataset


In [6]:
import numpy as np 
import pandas as pd

In [4]:
import gensim
import json

In [7]:
website_text_df = pd.read_csv('paras-and-lines-website-scraped.csv')
website_text_df

,lines
0,With the widest range of financial services
1,Bajaj Finserv Limited deals with financial ser...
2,"Bajaj Finance Limited, is the lending and weal..."
3,Bajaj Finserv Limited is the holding company f...
4,"The two insurance companies, Bajaj Allianz Gen..."
...,...
199124,"PRE-APPROVED OFFERS ON FITNESS, TRAVEL, FASHION"
199125,Low Brokerage Demat and Trading Account
199126,"Home, Kitchen appliances & Furniture on EMI"
199127,Pre-approved category on Smartphones and Laptops


In [8]:
#Preprocessing the text obtained from scraping to convert them to tokens
website_text = website_text_df['lines'].apply(gensim.utils.simple_preprocess)
website_text

0         [with, the, widest, range, of, financial, serv...
1         [bajaj, finserv, limited, deals, with, financi...
2         [bajaj, finance, limited, is, the, lending, an...
3         [bajaj, finserv, limited, is, the, holding, co...
4         [the, two, insurance, companies, bajaj, allian...
                                ...                        
199124    [pre, approved, offers, on, fitness, travel, f...
199125       [low, brokerage, demat, and, trading, account]
199126      [home, kitchen, appliances, furniture, on, emi]
199127    [pre, approved, category, on, smartphones, and...
199128    [loan, against, property, for, chartered, acco...
Name: lines, Length: 199129, dtype: object

In [9]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4,
)

In [10]:
#Building vocabulary for the model
model.build_vocab(website_text, progress_per=1000)

In [11]:
#Training the word2vec model
model.train(website_text, total_examples=model.corpus_count, epochs=model.epochs)


(13768649, 18025075)

In [95]:
model.wv.most_similar('travel')

[('bookings', 0.6554956436157227),
 ('tickets', 0.6472042798995972),
 ('holiday', 0.6426656246185303),
 ('flight', 0.6423884630203247),
 ('eyewear', 0.5996072888374329),
 ('hotel', 0.5946332812309265),
 ('trip', 0.5797046422958374),
 ('flights', 0.5565446615219116),
 ('vacation', 0.5495190620422363),
 ('packages', 0.5494423508644104)]

In [13]:
urls_df = pd.read_csv('webpage-urls-to-recommend-from.csv')
urls_df

,links
0,https://www.bajajfinserv.in/insights/best-mobi...
1,https://www.bajajfinserv.in/insights/best-mobi...
2,https://www.bajajfinserv.in/insights/top-5-inv...
3,https://www.bajajfinserv.in/insights/teeth-bra...
4,https://www.bajajfinserv.in/insights/mediclaim...
...,...
2049,https://www.bajajfinserv.in/insights/Indemnity...
2050,https://www.bajajfinserv.in/insights/Why-shoul...
2051,https://www.bajajfinserv.in/insights/What-is-p...
2052,https://www.bajajfinserv.in/insights/4-things-...


In [96]:
query_keyword = 'travel'
for link in urls_df['links']:
    heading_in_url = link[100:]
    words_in_url = heading_in_url.split('-')
    for word in words_in_url:
        if word.lower() in model.wv.key_to_index:
            if model.wv.similarity(word.lower(), query_keyword) > 0.5:
                print(link[37:].replace('-', ' '))
                print(link[:200])
                print()
            break

flexi personal loan or a term loan what do i choose if i want to travel
https://www.bajajfinserv.in/insights/flexi-personal-loan-or-a-term-loan-what-do-i-choose-if-i-want-to-travel

sights/everything you need to know about taking a loan for your wedding
https://cont-sites.bajajfinserv.in/insights/everything-you-need-to-know-about-taking-a-loan-for-your-wedding



In [84]:
links = urls_df['links']

In [65]:
model.wv.most_similar('emi')

[('emis', 0.6632044315338135),
 ('instalments', 0.4853419363498688),
 ('elica', 0.42314350605010986),
 ('swipe', 0.39923930168151855),
 ('electronics', 0.3897864520549774),
 ('converting', 0.36741340160369873),
 ('led', 0.3463621139526367),
 ('modular', 0.3396669626235962),
 ('convert', 0.3384273648262024),
 ('conveniently', 0.3345755636692047)]

In [17]:
model.save('website.model')

In [18]:
model.wv.most_similar('loan')

[('loans', 0.7145527601242065),
 ('sanction', 0.5098801851272583),
 ('lender', 0.46249842643737793),
 ('buyout', 0.43983882665634155),
 ('funding', 0.41831231117248535),
 ('funds', 0.40471693873405457),
 ('debt', 0.3807753026485443),
 ('approval', 0.37304672598838806),
 ('financing', 0.371757835149765),
 ('interest', 0.3669756352901459)]

In [20]:
model.wv.most_similar('card')

[('cards', 0.627054750919342),
 ('cardholders', 0.4902092516422272),
 ('linking', 0.4631918668746948),
 ('scorea', 0.40656518936157227),
 ('cheque', 0.4060649871826172),
 ('inline', 0.3979335427284241),
 ('necs', 0.39280810952186584),
 ('cardholder', 0.38754013180732727),
 ('transact', 0.3818022906780243),
 ('bureaus', 0.3735854923725128)]